In [22]:
import numpy as np
import tensorflow as tf
data = np.loadtxt('Buettner2015Features@1024.csv', delimiter=',', dtype='object')
gene_names = data[0, 1:]
data = data[1:, 1:].astype(np.float32)
print(data.shape)
data = tf.constant(data)
cov = tf.linalg.matmul(tf.transpose(data), data) / (data.shape[0] - 1)

(288, 1024)


In [2]:
import combination_n
import numpy as np

In [6]:
import imp
imp.reload(combination_n)

<module 'combination_n' from '/home/ringwalt/stochsparsepca/combination_n.cpython-310-x86_64-linux-gnu.so'>

In [9]:
combination_n.combination_3_batch(np.asarray([1, 3, 4]), 7, 10)

(array([[1, 3, 4],
        [1, 3, 5],
        [1, 3, 6],
        [1, 4, 5],
        [1, 4, 6],
        [1, 5, 6],
        [2, 3, 4],
        [2, 3, 5],
        [2, 3, 6],
        [2, 4, 5]], dtype=int32),
 array([2, 4, 5]))

In [3]:
import tensorflow as tf
@tf.function(jit_compile = True)
def gather_cov(cov, batches):
    inds_row = batches[..., :, None, None]
    inds_col = batches[..., None, :, None]
    inds_shape = tf.broadcast_dynamic_shape(
        tf.shape(inds_row), tf.shape(inds_col))
    inds = tf.concat([tf.broadcast_to(inds_row, inds_shape), tf.broadcast_to(inds_col, inds_shape)], axis=-1)
    return tf.gather_nd(cov, inds)

In [138]:
@tf.function
def evaluate_batch(cov, batch, cutoff):
    matrices = gather_cov(cov, batch)
    evalues, evectors = tf.linalg.eigh(matrices)
    pc0 = tf.math.reduce_max(evalues, axis=1)
    loc = tf.argmax(pc0)
    approx_entry = tf.math.reduce_mean(matrices, axis=2)
    approx_entry, _ = tf.linalg.normalize(approx_entry)
    approx_ev = tf.linalg.matmul(matrices, approx_entry[:, :, None])
    approx_ev, _ = tf.linalg.normalize(approx_ev, axis=1)
    approx_ev = tf.linalg.matmul(matrices, approx_ev)
    approx_ev = tf.linalg.norm(approx_ev[:, :, 0], axis=-1)
    return batch[loc], pc0[loc], pc0, approx_ev

In [7]:
@tf.function(reduce_retracing = True)
def evaluate_batch(cov, batch, cutoff):
    k = batch.shape[-1]
    matrices = gather_cov(cov, batch)
    matrices = tf.ensure_shape(matrices, [None, k, k])
    approx_entry = tf.math.reduce_mean(matrices, axis=2)
    approx_entry, _ = tf.linalg.normalize(approx_entry)
    approx_ev = tf.linalg.matmul(matrices, approx_entry[:, :, None])
    approx_ev, _ = tf.linalg.normalize(approx_ev, axis=1)
    approx_ev = tf.linalg.matmul(matrices, approx_ev)
    approx_ev = tf.linalg.norm(approx_ev[:, :, 0], axis=-1)
    keep_elem = approx_ev > cutoff
    matrices = tf.boolean_mask(matrices, keep_elem)
    batch_elem = tf.boolean_mask(tf.range(tf.shape(batch)[0]), keep_elem)
    evalues, evectors = tf.linalg.eigh(matrices)
    def evaluate():
        pc0 = tf.math.reduce_max(evalues, axis=1)
        loc = tf.argmax(pc0)
        return batch[batch_elem[loc]], pc0[loc]
    return tf.cond(tf.shape(matrices)[0] > 0, evaluate, lambda: (tf.zeros([k], tf.int32), tf.constant(np.nan)))

In [125]:
state = np.arange(3)
batch = combination_n.combination_3_batch(state, 100, 128)[0]
state

array([ 0,  2, 32])

In [146]:
evaluate_batch(cov, tf.constant(batch), 2.2
            )

(<tf.Tensor: shape=(3,), dtype=int32, numpy=array([ 0,  1, 80], dtype=int32)>,
 <tf.Tensor: shape=(), dtype=float32, numpy=2.2042725>)

In [89]:
cov.shape

TensorShape([512, 512])

In [5]:
%%time
pc0_stats = []
cutoff = 0.
state = np.arange(3)
while state is not None:
    batch, state = combination_n.combination_3_batch(state, 50, 1024 * 32)
    inds, score = evaluate_batch(cov, tf.constant(batch), tf.constant(cutoff))
    pc0_stats.append([inds, score])
    optimum, optimum_score = max(pc0_stats, key=lambda elem: elem[1].numpy())
    optimum = optimum.numpy()
    optimum_score = optimum_score.numpy()
    cutoff = tf.constant(optimum_score * 0.8, tf.float32)

2022-10-27 08:56:29.874899: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x7f52300091b0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2022-10-27 08:56:29.874933: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): NVIDIA T1000 8GB, Compute Capability 7.5
2022-10-27 08:56:29.890148: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-10-27 08:56:29.912354: I tensorflow/compiler/jit/xla_compilation_cache.cc:476] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
2022-10-27 08:56:29.915992: I tensorflow/core/util/cuda_solvers.cc:179] Creating GpuSolver handles for stream 0x1b5706e0


CPU times: user 1.96 s, sys: 449 ms, total: 2.41 s
Wall time: 2.43 s


In [ ]:
def greedy(n, k):
    cov_block.numpy().copy()

In [18]:
%%time
pc0_stats = []
cutoff = 0.
state = np.arange(3)
while state is not None:
    batch, state = combination_n.combination_3_batch(state, 512, 1024 * 32)
    inds, score = evaluate_batch(cov, tf.constant(batch), tf.constant(cutoff))
    pc0_stats.append([inds.numpy(), score.numpy()])
    optimum, optimum_score = max(pc0_stats, key=lambda elem: elem[1])
    cutoff = tf.constant(optimum_score * 0.8, tf.float32)

CPU times: user 57.1 s, sys: 13.3 s, total: 1min 10s
Wall time: 1min 9s


In [19]:
inds, cutoff, optimum_score

(<tf.Tensor: shape=(3,), dtype=int32, numpy=array([472, 487, 500], dtype=int32)>,
 <tf.Tensor: shape=(), dtype=float32, numpy=2.3107011>,
 2.8883765)

In [31]:
len(batch)

22694

In [33]:
%%time
evaluate_batch(cov, tf.constant(batch))
None

CPU times: user 2 s, sys: 472 ms, total: 2.47 s
Wall time: 2.46 s


In [39]:
(tf.linalg.trace(gather_cov(cov, tf.constant(batch))) < optimum_score).numpy().sum()

0

In [40]:
optimum_cov = gather_cov(cov, optimum)

In [42]:
tf.linalg.eigh(optimum_cov)

(<tf.Tensor: shape=(3,), dtype=float32, numpy=array([0.00840229, 0.11367464, 2.8883765 ], dtype=float32)>,
 <tf.Tensor: shape=(3, 3), dtype=float32, numpy=
 array([[-0.04780961,  0.8227641 ,  0.5663685 ],
        [-0.6839481 , -0.44020492,  0.5817512 ],
        [ 0.7279623 , -0.3595534 ,  0.58377403]], dtype=float32)>)

In [43]:
optimum_cov

<tf.Tensor: shape=(3, 3), dtype=float32, numpy=
array([[1.0034844 , 0.91078204, 0.921067  ],
       [0.91078204, 1.0034844 , 0.994734  ],
       [0.921067  , 0.994734  , 1.0034844 ]], dtype=float32)>

In [46]:
%%time
pc0_stats_16_100 = []
state = np.arange(16)
while state is not None:
    batch, state = combination_n.combination_3_batch(state, 100, 1024 * 64)
    inds, score, approx_score = evaluate_batch(cov, tf.constant(batch))
    pc0_stats.append([inds, score, approx_score])
optimum_100_16, optimum_score_100_16 = max(pc0_stats_16_100, key=lambda elem: elem[1].numpy())
optimum_100_16 = optimum_100_16.numpy()
optimum_score_100_16 = optimum_score_100_16.numpy()
optimum_100_16, optimum_score_100_16

KeyboardInterrupt: 